In [1]:
import torch
from sklearn.datasets import fetch_openml

from neural_blueprints.utils import Trainer, infer_types
from neural_blueprints.config.architectures import TabularBERTConfig
from neural_blueprints.config.utils import TrainerConfig
from neural_blueprints.architectures import TabularBERT
from neural_blueprints.datasets import MaskedTabularDataset, TabularSingleLabelDataset
from neural_blueprints.preprocess import TabularPreprocessor

import logging
logging.basicConfig(
    level=logging.DEBUG,  # or DEBUG if you want even more detail
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s"
)

In [2]:
data = fetch_openml(name="adult", version=2, as_frame=True)
X = data.data
y = data.target

data = X.copy()
data['income'] = y

dtypes = infer_types(data)
data = data.astype(dtypes)
data.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,NaN,103497,Some-college,10,Never-married,NaN,Own-child,White,Female,0,0,30,United-States,<=50K


In [3]:
preprocessor = TabularPreprocessor()
data, discrete_features, continuous_features = preprocessor.run(data)

2025-12-27 17:04:14,961 - neural_blueprints.preprocess.tabular_preprocess - INFO - Identified 10 discrete features: ['workclass', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country', 'income']
2025-12-27 17:04:14,961 - neural_blueprints.preprocess.tabular_preprocess - INFO - Identified 5 continuous features: ['age', 'fnlwgt', 'capital-gain', 'capital-loss', 'hours-per-week']


### Income Inference Accuracy

In [4]:
dataset = TabularSingleLabelDataset(
    data=data,
    label_column='income',              # Specify the label column for single-label classification
    discrete_features=discrete_features,
    continuous_features=continuous_features
)

train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

In [5]:
bert_config = TabularBERTConfig(
    input_cardinalities = dataset.cardinalities,
    output_cardinalities = [2],
    latent_dim = 32,
    encoder_layers=8,
    dropout_p = 0.2,
    normalization = "batchnorm1d",
    activation = "gelu",
    final_activation = None
)

model = TabularBERT(bert_config)
model.blueprint()

TabularBERT(
  (input_projection): TabularInputProjection(
    (input_projections): ModuleList(
      (0): FeedForwardNetwork(
        (network): Sequential(
          (0): DenseLayer(
            (layer): Sequential(
              (0): Linear(in_features=1, out_features=128, bias=True)
              (1): NormalizationLayer(
                (network): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              )
              (2): GELU(approximate='none')
              (3): DropoutLayer(
                (dropout): Dropout(p=0.2, inplace=False)
              )
            )
          )
          (1): DenseLayer(
            (layer): Sequential(
              (0): Linear(in_features=128, out_features=64, bias=True)
              (1): NormalizationLayer(
                (network): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              )
              (2): GELU(approximate='none')
              (3): DropoutLayer(
   

TabularBERTConfig(input_cardinalities=[1, 9, 1, 16, 16, 7, 15, 6, 5, 2, 1, 1, 1, 42], output_cardinalities=[2], latent_dim=32, encoder_layers=8, dropout_p=0.2, normalization='batchnorm1d', activation='gelu', final_activation=None)

In [6]:
trainer = Trainer(
    model=model,
    config=TrainerConfig(
        criterion='cross_entropy',
        optimizer='adam',
        early_stopping_patience=2,
        learning_rate=1e-3,
        weight_decay=1e-5,
        save_weights_path="./models/bert_adult.pth",
        batch_size=128
    )
)
trainer.train(train_dataset, val_dataset, epochs=5, visualize=True)

2025-12-27 17:04:15,486 - neural_blueprints.utils.trainer - INFO - Trainer initialized on device: cpu


Directory ./models already exists. Existing weights are overwritten.


Training Epochs:  20%|██        | 1/5 [00:22<01:30, 22.61s/epoch]

Epoch 1/5, Training Loss: 0.7340, Validation Loss: 0.6914


Training Epochs:  40%|████      | 2/5 [00:45<01:08, 22.68s/epoch]

...

Epoch 3/5, Training Loss: 0.6823, Validation Loss: 0.6884


Training Epochs:  80%|████████  | 4/5 [01:31<00:22, 22.88s/epoch]

Epoch 4/5, Training Loss: 0.6810, Validation Loss: 0.6833


Training Epochs: 100%|██████████| 5/5 [01:53<00:00, 22.77s/epoch]
2025-12-27 17:06:09,325 - neural_blueprints.utils.trainer - INFO - Training completed in 113.84 seconds.
2025-12-27 17:06:09,325 - neural_blueprints.utils.trainer - INFO - Best validation loss: 6.8334e-01


Epoch 5/5, Training Loss: 0.6796, Validation Loss: 0.6839


In [7]:
trainer.predict(val_dataset)

2025-12-27 17:06:10,131 - neural_blueprints.utils.trainer - INFO - Inference completed in 0.63 seconds.


Predictions: tensor([1, 2, 2, 1, 1]), 
 Ground Truth: tensor([1, 1, 2, 1, 1])
Prediction Accuracy: 0.8502


np.float64(0.8501535312180143)

### Masked Dataset Inference Accuracy

In [8]:
# Create dataset
dataset = MaskedTabularDataset(
    data, 
    discrete_features, 
    continuous_features,
    mask_prob=0.35
)

train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

In [9]:
bert_config = TabularBERTConfig(
    input_cardinalities = dataset.cardinalities,   
    latent_dim = 32,
    encoder_layers=8,
    dropout_p = 0.2,
    normalization = "batchnorm1d",
    activation = "gelu",
    final_activation = None
)

model = TabularBERT(bert_config)
model.blueprint()

TabularBERT(
  (input_projection): TabularInputProjection(
    (input_projections): ModuleList(
      (0): FeedForwardNetwork(
        (network): Sequential(
          (0): DenseLayer(
            (layer): Sequential(
              (0): Linear(in_features=1, out_features=128, bias=True)
              (1): NormalizationLayer(
                (network): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              )
              (2): GELU(approximate='none')
              (3): DropoutLayer(
                (dropout): Dropout(p=0.2, inplace=False)
              )
            )
          )
          (1): DenseLayer(
            (layer): Sequential(
              (0): Linear(in_features=128, out_features=64, bias=True)
              (1): NormalizationLayer(
                (network): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              )
              (2): GELU(approximate='none')
              (3): DropoutLayer(
   

TabularBERTConfig(input_cardinalities=[1, 9, 1, 16, 16, 7, 15, 6, 5, 2, 1, 1, 1, 42, 2], output_cardinalities=[1, 9, 1, 16, 16, 7, 15, 6, 5, 2, 1, 1, 1, 42, 2], latent_dim=32, encoder_layers=8, dropout_p=0.2, normalization='batchnorm1d', activation='gelu', final_activation=None)

In [10]:
trainer = Trainer(
    model=model,
    config=TrainerConfig(
        criterion='masked_reconstruction',
        optimizer='adam',
        early_stopping_patience=2,
        learning_rate=1e-3,
        weight_decay=1e-5,
        save_weights_path="./models/bert_adult.pth",
        batch_size=128
    )
)
trainer.train(train_dataset, val_dataset, epochs=5, visualize=True)

2025-12-27 17:06:10,171 - neural_blueprints.utils.trainer - INFO - Trainer initialized on device: cpu


Directory ./models already exists. Existing weights are overwritten.


Training Epochs:  20%|██        | 1/5 [00:28<01:54, 28.65s/epoch]

Epoch 1/5, Training Loss: 6.3588, Validation Loss: 28.5576


Training Epochs:  40%|████      | 2/5 [00:56<01:24, 28.03s/epoch]

...

Epoch 3/5, Training Loss: 6.0637, Validation Loss: 27.5188


Training Epochs:  80%|████████  | 4/5 [01:51<00:27, 27.94s/epoch]

Epoch 4/5, Training Loss: 6.0228, Validation Loss: 26.4797


Training Epochs: 100%|██████████| 5/5 [02:20<00:00, 28.07s/epoch]
2025-12-27 17:08:30,527 - neural_blueprints.utils.trainer - INFO - Training completed in 140.36 seconds.
2025-12-27 17:08:30,527 - neural_blueprints.utils.trainer - INFO - Best validation loss: 2.6454e+01


Epoch 5/5, Training Loss: 5.9954, Validation Loss: 26.4542


In [11]:
trainer.predict(val_dataset)

2025-12-27 17:08:31,248 - neural_blueprints.utils.inference.predict - INFO - Feature Column 0:
2025-12-27 17:08:31,249 - neural_blueprints.utils.inference.predict - INFO - Predicted attribute values: [0.36932698 0.35003644 0.37635037 0.41367605 0.36804762]
2025-12-27 17:08:31,249 - neural_blueprints.utils.inference.predict - INFO - True attribute values: [0.41095892 0.34246576 0.6849315  0.21917808 0.38356164]
2025-12-27 17:08:31,250 - neural_blueprints.utils.inference.predict - INFO - Accuracy: 0.2438

2025-12-27 17:08:31,250 - neural_blueprints.utils.inference.predict - INFO - Feature Column 1:
2025-12-27 17:08:31,251 - neural_blueprints.utils.inference.predict - WARNING - Warning: Only one class predicted for this feature.
2025-12-27 17:08:31,251 - neural_blueprints.utils.inference.predict - INFO - Predicted attribute values: [4 4 4 4 4]
2025-12-27 17:08:31,251 - neural_blueprints.utils.inference.predict - INFO - True attribute values: [4. 4. 4. 7. 4.]
2025-12-27 17:08:31,252 - neur

{'avg_discrete_accuracy': np.float64(0.6606882143723933),
 'avg_continuous_accuracy': np.float64(0.6389784808074499),
 'overall_avg_accuracy': np.float64(0.6534516365174121)}